In [1060]:
import numpy as np
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation
import Levenshtein

In [1061]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[~df.str.contains('think|should|maybe', case = False)]
df = df[df.str.contains('best', case = False)]
df = df.str.replace('http\S+|www.\S+', '', case=False,regex=True)
best_df = df.str.replace('#GoldenGlobes|#GoldenGlobe|Golden Globe|Golden Globes', "", case = False, regex = True)

if best_df.shape[0] >5000:
    best_df = best_df.sample(n=5000)
best_df.shape[0]

5000

# pattern2 = [{"ORTH":'Best'}, {"POS":{"IN":['NOUN','PROPN','ADP','DET']}, 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"DEP":{"IN":['compound','nsubj','cc','conj','dobj']}}]

In [1062]:
pattern = [{"LOWER":'best'}, {"DEP":{"IN":['compound','nmod','dobj','prep','det','pobj','amod','nsubj','ROOT']}, 'OP':'+'},{'ORTH': '-', 'OP':'?'}, {"POS": {"IN": ['PROPN','NOUN','CCONJ','ADJ']}, 'OP':'*'}]
# pattern = [{"LOWER":'best','OP':'+'}, {"POS":{"IN":['NOUN','PROPN','ADP','DET','ADJ']}, 'OP':'+'}, {'ORTH': '-', 'OP':'?'}, {"POS": {"IN": ['PROPN','NOUN','CCONJ','ADJ']}, 'OP':'*'}]

In [1063]:
pattern2 = [{"ENT_TYPE":{"IN":['PERSON','WORK_OF_ART']},'OP':'+'}]

In [1064]:
matcher = Matcher(nlp.vocab)
matcher.add('Winner', [pattern])

In [1065]:
matcher2 = Matcher(nlp.vocab)
matcher2.add('Deleter',[pattern2])

In [1066]:
doc = nlp('he wins best motion picture - comedy or musical for')

In [1067]:
matches = matcher(doc)
matches[-1][2]
print(len(matches))
matches

6


[(7971871675768048331, 2, 4),
 (7971871675768048331, 2, 5),
 (7971871675768048331, 2, 6),
 (7971871675768048331, 2, 7),
 (7971871675768048331, 2, 8),
 (7971871675768048331, 2, 9)]

In [1068]:
for entity in doc:
    print(entity.text, entity.pos_,entity.dep_,entity.lemma_)

he PRON nsubj -PRON-
wins VERB ROOT win
best ADJ amod good
motion NOUN compound motion
picture NOUN compound picture
- PUNCT punct -
comedy NOUN dobj comedy
or CCONJ cc or
musical ADJ conj musical
for ADP prep for


In [1069]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(match_id, string_id, start, end, span.text)

7971871675768048331 Winner 2 4 best motion
7971871675768048331 Winner 2 5 best motion picture
7971871675768048331 Winner 2 6 best motion picture -
7971871675768048331 Winner 2 7 best motion picture - comedy
7971871675768048331 Winner 2 8 best motion picture - comedy or
7971871675768048331 Winner 2 9 best motion picture - comedy or musical


In [1070]:
awards_list = ["best screenplay - motion picture", 
            "best director - motion picture", 
            "best performance by an actress in a television series - comedy or musical",
            "best foreign language film",
            "best performance by an actor in a supporting role in a motion picture",
            "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
            "best motion picture - comedy or musical", 
            "best performance by an actress in a motion picture - comedy or musical", 
            "Best mini-series or motion picture made for television", "best original score - motion picture", 
            "best performance by an actress in a television series - drama", 
            "best performance by an actress in a motion picture - drama",
            "cecil b. demille award",
            "best performance by an actor in a motion picture - comedy or musical", "best motion picture - drama",
            "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television", 
            "best performance by an actress in a supporting role in a motion picture",
            "best television series - drama", 
            "best performance by an actor in a mini-series or motion picture made for television",
            "best performance by an actress in a mini-series or motion picture made for television",
            "best animated feature film", "best original song - motion picture", 
            "best performance by an actor in a motion picture - drama", 
            "best television series - comedy or musical",
            "best performance by an actor in a television series - drama",
            "best performance by an actor in a television series - comedy or musical"]
print(len(awards_list))

26


In [1071]:
sum_val = 0
for text in awards_list:
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        print(span)
        sum_val += 1
    else:
        print('bad:' + text)
print(sum_val)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
bad:Best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
bad:cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series
best performance by an actress in a supporting role in a motion picture
best television series - drama
best performance by an actor in a mini-series or motion picture
best performance by 

In [1072]:
awards = {}
awards = defaultdict(lambda: 1, awards)
for i, text in best_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        awards[str(span).strip(punctuation).strip()] = awards[str(span).strip(punctuation).strip()] +1

In [1073]:
for key in list(awards):
    doc = nlp(key)
    matches = matcher2(doc)
    if len(matches) != 0:
        awards.pop(key)
awards = sorted(awards.items(), key=lambda item: item[1], reverse = True)
print(awards[0:100])

[('Best Original Song', 168), ('Best Motion Picture', 89), ('Best Director', 87), ('Best Motion Picture - Comedy or Musical', 71), ('Best Supporting Actress', 68), ('Best Motion Picture - Drama', 51), ('Best Actor in a Motion Picture - Comedy or Musical', 50), ('Best Screenplay', 49), ('Best Supporting Actress in a Motion Picture', 48), ('Best Actress in a Motion Picture - Comedy or Musical', 47), ('Best Television Series - Drama', 43), ('best acceptance speech', 43), ('Best Actress', 41), ('best picture', 41), ('Best Supporting Actress in a TV Movie', 37), ('Best Supporting Actor in a Motion Picture - Christoph Waltz', 35), ('Best Actor', 35), ('Best Actor in a Motion Picture', 33), ('Best Animated Feature Film - Brave', 33), ('Best TV Series', 32), ('Best Original Score - Mychael Danna', 32), ('Best Actress in a Motion Picture - Drama', 31), ('Best Buy', 30), ('BEST speech', 30), ('Best Actress in a Comedy Series', 29), ('Best Supporting Actor', 29), ('Best Actor in a Motion Picture 

In [1074]:
new_awards = []
for i in awards:
    if i[1] > 2:
        new_awards.append(i[0].lower())

In [1075]:
final_list = []
final_list.append(new_awards[0])
def distance(t1,r1):
    t = t1.split()
    r = r1.split()
    if len(t) >= len(r):
        long = t
        short = r
        ret = t1
    else:
        long = r
        short = t
        ret = r1
    dist = 0
    for word in short:
        min_val = float('inf')
        for word2 in long:
            min_val = min(min_val,Levenshtein.distance(word, word2))
        dist += min_val
    return dist,ret

In [1076]:
distance('best actor', 'best actor - comedy')

(0, 'best actor - comedy')

In [1077]:
print(final_list)

['best original song']


In [1078]:
for awrd in new_awards[1:]:
    similar = False
    for i,ans in enumerate(final_list):
        dist,long = distance(awrd,ans)
        if dist <= 1:
            similar = True
            final_list[i] = long
    if (not similar):
        final_list.append(awrd)
print(final_list)

['best original song win at the', 'best actor in a motion picture - comedy or musical', 'best director award for #argo at the', 'best supporting actress in a motion picture win for les miserables', 'best actress in a motion picture - drama', 'best screenplay - motion picture', 'best actress in a motion picture - comedy or musical', 'best television series actor - drama', 'best acceptance speech', 'best supporting actress in a tv movie', 'best supporting actor in a motion picture - christoph waltz', 'best animated feature film - brave', 'best performance by an actress in a tv series - musical or comedy', 'best original score - mychael danna', 'best show on tv by miles', 'best performance by an actress in a tv series - musical or comedy', 'best performance by an actor in a motion picture - drama', 'best foreign film - amour', 'best original song win at the', 'best actress in a miniseries or tv movie', 'best actress award', 'best s opening', 'best actress in a television series - drama', 

In [1080]:
print(final_list[0:26])

['best original song win at the', 'best actor in a motion picture - comedy or musical', 'best director award for #argo at the', 'best supporting actress in a motion picture win for les miserables', 'best actress in a motion picture - drama', 'best screenplay - motion picture', 'best actress in a motion picture - comedy or musical', 'best television series actor - drama', 'best acceptance speech', 'best supporting actress in a tv movie', 'best supporting actor in a motion picture - christoph waltz', 'best animated feature film - brave', 'best performance by an actress in a tv series - musical or comedy', 'best original score - mychael danna', 'best show on tv by miles', 'best performance by an actress in a tv series - musical or comedy', 'best performance by an actor in a motion picture - drama', 'best foreign film - amour', 'best original song win at the', 'best actress in a miniseries or tv movie', 'best actress award', 'best s opening', 'best actress in a television series - drama', 

In [1081]:
answer = list(dict.fromkeys(final_list))
print(answer[0:26])

['best original song win at the', 'best actor in a motion picture - comedy or musical', 'best director award for #argo at the', 'best supporting actress in a motion picture win for les miserables', 'best actress in a motion picture - drama', 'best screenplay - motion picture', 'best actress in a motion picture - comedy or musical', 'best television series actor - drama', 'best acceptance speech', 'best supporting actress in a tv movie', 'best supporting actor in a motion picture - christoph waltz', 'best animated feature film - brave', 'best performance by an actress in a tv series - musical or comedy', 'best original score - mychael danna', 'best show on tv by miles', 'best performance by an actor in a motion picture - drama', 'best foreign film - amour', 'best actress in a miniseries or tv movie', 'best actress award', 'best s opening', 'best actress in a television series - drama', 'best motion picture drama goes to argo', 'best miniseries or motion picture', 'best actor in a televi